# Using Macauff Cross-match results in LSDB

This notebook goes over loading an association table that has been imported from a macauff cross-match result, using it to join lsdb catalogs, and performing queries on the joined datasets.

First, the necesarry imports and setting up a Dask Client

In [1]:
import lsdb

from dask.distributed import Client

In [2]:
client = Client(n_workers=6, memory_limit='20GB', local_directory="/data3/epyc/projects3/sean_hipscat", dashboard_address=':33725', threads_per_worker=1)
client

/astro/users/seanmcgu/anaconda3/envs/lsdb/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 33725 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41247 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41247/status,
Dashboard: http://127.0.0.1:41247/status,Workers: 6
Total threads: 6,Total memory: 111.76 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36242,Workers: 6
Dashboard: http://127.0.0.1:41247/status,Total threads: 6
Started: Just now,Total memory: 111.76 GiB
Comm: tcp://127.0.0.1:41450,Total threads: 1
Dashboard: http://127.0.0.1:33535/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:33699,


## Loading the catalogs

Each catalog is loaded using the `lsdb.read_hipscat` method. First the macauff association table, and then the gaia and wise object catalogs. The loading is done lazily, so at this stage the results show just the column names and types.

In [9]:
macauff_association = lsdb.read_hipscat("/data3/epyc/data3/hipscat/test_catalogs/macauff_association/")
macauff_association

,gaia_source_id,gaia_ra,gaia_dec,BP,G,RP,catwise_name,catwise_ra,catwise_dec,W1,W2,match_p,separation,eta,xi,gaia_avg_cont,catwise_avg_cont,gaia_cont_f1,gaia_cont_f10,catwise_cont_f1,catwise_cont_f10,catwise_fit_sig,Norder,Dir,Npix,join_Norder,join_Dir,join_Npix
npartitions=3933,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64
72057594037927936,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13817043656772681728,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18446744073709551615,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
gaia = lsdb.read_hipscat("/data3/epyc/data3/hipscat/test_catalogs/gaia_symbolic")
gaia

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Npix,Dir
npartitions=3933,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,bool,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,bool,bool,int64,bool,bool,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,flo

In [11]:
catwise = lsdb.read_hipscat("/epyc/projects3/sam_hipscat/catwise2020/catwise2020")
catwise

,index,source_name,source_id,ra,dec,sigra,sigdec,sigradec,wx,wy,w1sky,w1sigsk,w1conf,w2sky,w2sigsk,w2conf,w1fitr,w2fitr,w1snr,w2snr,w1flux,w1sigflux,w2flux,w2sigflux,w1mpro,w1sigmpro,w1rchi2,w2mpro,w2sigmpro,w2rchi2,rchi2,nb,na,w1Sat,w2Sat,w1mag,w1sigm,w1flg,w1Cov,w2mag,w2sigm,w2flg,w2Cov,w1mag_1,w1sigm_1,w1flg_1,w2mag_1,w2sigm_1,w2flg_1,w1mag_2,w1sigm_2,w1flg_2,w2mag_2,w2sigm_2,w2flg_2,w1mag_3,w1sigm_3,w1flg_3,w2mag_3,w2sigm_3,w2flg_3,w1mag_4,w1sigm_4,w1flg_4,w2mag_4,w2sigm_4,w2flg_4,w1mag_5,w1sigm_5,w1flg_5,w2mag_5,w2sigm_5,w2flg_5,w1mag_6,w1sigm_6,w1flg_6,w2mag_6,w2sigm_6,w2flg_6,w1mag_7,w1sigm_7,w1flg_7,w2mag_7,w2sigm_7,w2flg_7,w1mag_8,w1sigm_8,w1flg_8,w2mag_8,w2sigm_8,w2flg_8,w1NM,w1M,w1magP,w1sigP1,w1sigP2,w1k,w1Ndf,w1mLQ,w1mJDmin,w1mJDmax,w1mJDmean,w2NM,w2M,w2magP,w2sigP1,w2sigP2,w2k,w2Ndf,w2mLQ,w2mJDmin,w2mJDmax,w2mJDmean,rho12,q12,nIters,nSteps,mdetID,p1,p2,MeanObsMJD,ra_pm,dec_pm,sigra_pm,sigdec_pm,sigradec_pm,PMRA,PMDec,sigPMRA,sigPMDec,w1snr_pm,w2snr_pm,w1flux_pm,w1sigflux_pm,w2flux_pm,w2sigflux_pm,w1mpro_pm,w1sigmpro_pm,w1rchi2_pm,w2mpro_pm,w2sigmpro_pm,w2rchi2_pm,rchi2_pm,pmcode,nIters_pm,nSteps_pm,dist,dw1mag,rch2w1,dw2mag,rch2w2,elon_avg,elonSig,elat_avg,elatSig,Delon,DelonSig,Delat,DelatSig,DelonSNR,DelatSNR,chi2pmra,chi2pmdec,ka,k1,k2,km,par_pm,par_pmSig,par_stat,par_sigma,dist_x,cc_flags,w1cc_map,w1cc_map_str,w2cc_map,w2cc_map_str,n_aw,ab_flags,w1ab_map,w1ab_map_str,w2ab_map,w2ab_map_str,glon,glat,elon,elat,unwise_objid,Norder,Dir,Npix
npartitions=4080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,Int64,Int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,object,Int64,object,Int64,object,int64,object,int64,object,int64,object,float64,float64,float64,float64,object,int32,int32,int32
4503599627370496,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

## Joining using the Association table

The join is performed from gaia to catwise, using the `catalog.join` method, specifying the macauff association table to use to join through. This, like most LSDB operations, is also done lazily, and so the result is again a table schema with the gaia columns, the extra columns from the macauff table such as the probabilities and separations, and the wise columns.

To perform the computation, the `compute()` method is used to load the data and perform the join. In this case, we only compute a single partition so the result can be returned into memory. To run over the entire dataset and save to disk, the `to_hipscat(path)` method can be used on the `joined` catalog.

In [12]:
joined = gaia.join(catwise, through=macauff_association)
joined

,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,parallax_error_gaia,parallax_over_error_gaia,pm_gaia,pmra_gaia,pmra_error_gaia,pmdec_gaia,pmdec_error_gaia,ra_dec_corr_gaia,ra_parallax_corr_gaia,ra_pmra_corr_gaia,ra_pmdec_corr_gaia,dec_parallax_corr_gaia,dec_pmra_corr_gaia,dec_pmdec_corr_gaia,parallax_pmra_corr_gaia,parallax_pmdec_corr_gaia,pmra_pmdec_corr_gaia,astrometric_n_obs_al_gaia,astrometric_n_obs_ac_gaia,astrometric_n_good_obs_al_gaia,astrometric_n_bad_obs_al_gaia,astrometric_gof_al_gaia,astrometric_chi2_al_gaia,astrometric_excess_noise_gaia,astrometric_excess_noise_sig_gaia,astrometric_params_solved_gaia,astrometric_primary_flag_gaia,nu_eff_used_in_astrometry_gaia,pseudocolour_gaia,pseudocolour_error_gaia,ra_pseudocolour_corr_gaia,dec_pseudocolour_corr_gaia,parallax_pseudocolour_corr_gaia,pmra_pseudocolour_corr_gaia,pmdec_pseudocolour_corr_gaia,astrometric_matched_transits_gaia,visibility_periods_used_gaia,astrometric_sigma5d_max_gaia,matched_transits_gaia,new_matched_transits_gaia,matched_transits_removed_gaia,ipd_gof_harmonic_amplitude_gaia,ipd_gof_harmonic_phase_gaia,ipd_frac_multi_peak_gaia,ipd_frac_odd_win_gaia,ruwe_gaia,scan_direction_strength_k1_gaia,scan_direction_strength_k2_gaia,scan_direction_strength_k3_gaia,scan_direction_strength_k4_gaia,scan_direction_mean_k1_gaia,scan_direction_mean_k2_gaia,scan_direction_mean_k3_gaia,scan_direction_mean_k4_gaia,duplicated_source_gaia,phot_g_n_obs_gaia,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_g_mean_flux_over_error_gaia,phot_g_mean_mag_gaia,phot_bp_n_obs_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_bp_mean_flux_over_error_gaia,phot_bp_mean_mag_gaia,phot_rp_n_obs_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,phot_rp_mean_flux_over_error_gaia,phot_rp_mean_mag_gaia,phot_bp_rp_excess_factor_gaia,phot_bp_n_contaminated_transits_gaia,phot_bp_n_blended_transits_gaia,phot_rp_n_contaminated_transits_gaia,phot_rp_n_blended_transits_gaia,phot_proc_mode_gaia,bp_rp_gaia,bp_g_gaia,g_rp_gaia,radial_velocity_gaia,radial_velocity_error_gaia,rv_method_used_gaia,rv_nb_transits_gaia,rv_nb_deblended_transits_gaia,rv_visibility_periods_used_gaia,rv_expected_sig_to_noise_gaia,rv_renormalised_gof_gaia,rv_chisq_pvalue_gaia,rv_time_duration_gaia,rv_amplitude_robust_gaia,rv_template_teff_gaia,rv_template_logg_gaia,rv_template_fe_h_gaia,rv_atm_param_origin_gaia,vbroad_gaia,vbroad_error_gaia,vbroad_nb_transits_gaia,grvs_mag_gaia,grvs_mag_error_gaia,grvs_mag_nb_transits_gaia,rvs_spec_sig_to_noise_gaia,phot_variable_flag_gaia,l_gaia,b_gaia,ecl_lon_gaia,ecl_lat_gaia,in_qso_candidates_gaia,in_galaxy_candidates_gaia,non_single_star_gaia,has_xp_continuous_gaia,has_xp_sampled_gaia,has_rvs_gaia,has_epoch_photometry_gaia,has_epoch_rv_gaia,has_mcmc_gspphot_gaia,has_mcmc_msc_gaia,in_andromeda_survey_gaia,classprob_dsc_combmod_quasar_gaia,classprob_dsc_combmod_galaxy_gaia,classprob_dsc_combmod_star_gaia,teff_gspphot_gaia,teff_gspphot_lower_gaia,teff_gspphot_upper_gaia,logg_gspphot_gaia,logg_gspphot_lower_gaia,logg_gspphot_upper_gaia,mh_gspphot_gaia,mh_gspphot_lower_gaia,mh_gspphot_upper_gaia,distance_gspphot_gaia,distance_gspphot_lower_gaia,distance_gspphot_upper_gaia,azero_gspphot_gaia,azero_gspphot_lower_gaia,azero_gspphot_upper_gaia,ag_gspphot_gaia,ag_gspphot_lower_gaia,ag_gspphot_upper_gaia,ebpminrp_gspphot_gaia,ebpminrp_gspphot_lower_gaia,ebpminrp_gspphot_upper_gaia,libname_gspphot_gaia,Norder_gaia,Npix_gaia,Dir_gaia,gaia_ra,gaia_dec,BP,G,RP,catwise_ra,catwise_dec,W1,W2,match_p,separation,eta,xi,gaia_avg_cont,catwise_avg_cont,gaia_cont_f1,gaia_cont_f10,catwise_cont_f1,catwise_cont_f10,catwise_fit_sig,index_catwise2020,source_name_catwise2020,source_id_catwise2020,ra_catwise2020,dec_catwise2020,sigra_catwise2020,sigdec_catwise2020,sigradec_catwise2020,wx_catwise2020,wy_catwise2020,w1sky_catwise2020,w1sigsk_catwise2020,w1conf_catwise2020,w2sky_catwise2020,w2sigsk_catwise20

In [13]:
joined._ddf.partitions[0].compute()

,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,...,w2ab_map_catwise2020,w2ab_map_str_catwise2020,glon_catwise2020,glat_catwise2020,elon_catwise2020,elat_catwise2020,unwise_objid_catwise2020,Norder_catwise2020,Dir_catwise2020,Npix_catwise2020
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
2723009265664,1636148068921376768,Gaia DR3 4295806720,4295806720,545300884,2016.0,44.996155,0.101618,0.005615,0.101334,0.354331,...,0,-999,176.951030,-48.901533,42.533685,-16.329547,0453p000o0012311r02,4,0,0
2744484102144,1636148068921376768,Gaia DR3 34361129088,34361129088,894504938,2016.0,45.004320,0.097320,0.021048,0.101753,3.235017,...,0,-999,176.942772,-48.884898,42.546586,-16.317181,0453p000o0027863r02,4,0,0
2787433775104,1636148068921376768,Gaia DR3 38655544960,38655544960,1757259052,2016.0,45.004978,0.017885,0.019880,0.018772,3.139170,...,0,-999,176.944733,-48.885293,42.546827,-16.318509,0453p000o0012539r02,4,0,0
3062311682048,1636148068921376768,Gaia DR3 309238066432,309238066432,327609758,2016.0,44.995037,0.322039,0.038152,0.283505,1.383150,...,0,-999,176.914190,-48.879785,42.542475,-16.298105,0453p000o0012834r02,4,0,0
3277060046848,1636148068921376768,Gaia DR3 343597448960,343597448960,1791600903,2016.0,44.963896,0.117176,0.043595,0.109004,0.196148,...,0,-999,176.875441,-48.898384,42.513178,-16.283812,0453p000o0012911r02,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4503011216850944,1636148068921376768,Gaia DR3 2250631582599680,2250631582599680,1371771118,2016.0,45.041765,0.139761,4.724431,0.133588,0.960993,...,0,-999,172.148063,-45.490376,43.989802,-11.826674,0455p045o0013201r02,4,0,0
4503071346393088,1636148068921376768,Gaia DR3 2250734661812352,2250734661812352,641527459,2016.0,44.967614,0.117784,4.694361,0.107240,2.180210,...,0,-999,172.099346,-45.562511,43.908638,-11.834260,0455p045o0012696r02,4,0,0
4503376289071104,1636148068921376768,Gaia DR3 2250734661819392,2250734661819392,1549774147,2016.0,44.965569,0.448048,4.693979,0.385873,1.993614,...,0,-999,172.097860,-45.564139,43.906665,-11.834199,0455p045o0024348r02,4,0,0


## Querying the joined table

The catalog's `query()` method can be used to perform queries on the data. The query strings follow the pandas query function, for which the [full documentation is available here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html).

In this case, we query on the `match_p` column, getting rows with values over `0.999`.

In [14]:
joined.query("match_p > 0.999")._ddf.partitions[0].compute()

,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,...,w2ab_map_catwise2020,w2ab_map_str_catwise2020,glon_catwise2020,glat_catwise2020,elon_catwise2020,elat_catwise2020,unwise_objid_catwise2020,Norder_catwise2020,Dir_catwise2020,Npix_catwise2020
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
2723009265664,1636148068921376768,Gaia DR3 4295806720,4295806720,545300884,2016.0,44.996155,0.101618,0.005615,0.101334,0.354331,...,0,-999,176.951030,-48.901533,42.533685,-16.329547,0453p000o0012311r02,4,0,0
2744484102144,1636148068921376768,Gaia DR3 34361129088,34361129088,894504938,2016.0,45.004320,0.097320,0.021048,0.101753,3.235017,...,0,-999,176.942772,-48.884898,42.546586,-16.317181,0453p000o0027863r02,4,0,0
2787433775104,1636148068921376768,Gaia DR3 38655544960,38655544960,1757259052,2016.0,45.004978,0.017885,0.019880,0.018772,3.139170,...,0,-999,176.944733,-48.885293,42.546827,-16.318509,0453p000o0012539r02,4,0,0
3062311682048,1636148068921376768,Gaia DR3 309238066432,309238066432,327609758,2016.0,44.995037,0.322039,0.038152,0.283505,1.383150,...,0,-999,176.914190,-48.879785,42.542475,-16.298105,0453p000o0012834r02,4,0,0
3277060046848,1636148068921376768,Gaia DR3 343597448960,343597448960,1791600903,2016.0,44.963896,0.117176,0.043595,0.109004,0.196148,...,0,-999,176.875441,-48.898384,42.513178,-16.283812,0453p000o0012911r02,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4503011216850944,1636148068921376768,Gaia DR3 2250631582599680,2250631582599680,1371771118,2016.0,45.041765,0.139761,4.724431,0.133588,0.960993,...,0,-999,172.148063,-45.490376,43.989802,-11.826674,0455p045o0013201r02,4,0,0
4503071346393088,1636148068921376768,Gaia DR3 2250734661812352,2250734661812352,641527459,2016.0,44.967614,0.117784,4.694361,0.107240,2.180210,...,0,-999,172.099346,-45.562511,43.908638,-11.834260,0455p045o0012696r02,4,0,0
4503376289071104,1636148068921376768,Gaia DR3 2250734661819392,2250734661819392,1549774147,2016.0,44.965569,0.448048,4.693979,0.385873,1.993614,...,0,-999,172.097860,-45.564139,43.906665,-11.834199,0455p045o0024348r02,4,0,0


In [10]:
client.close()